In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install datasets

In [2]:
import pandas as pd

In [3]:
from datasets import load_dataset
df = load_dataset('ruslanmv/ai-medical-chatbot')

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [4]:
# Convert to Pandas DataFrame
df = pd.DataFrame(df['train'])

In [5]:
df

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...
...,...,...,...
256911,Why is hair fall increasing while using Bontre...,I am suffering from excessive hairfall. My doc...,"Hello Dear Thanks for writing to us, we are he..."
256912,Why was I asked to discontinue Androanagen whi...,"Hi Doctor, I have been having severe hair fall...","hello, hair4u is combination of minoxid..."
256913,Can Mintop 5% Lotion be used by women for seve...,Hi..i hav sever hair loss problem so consulted...,HI I have evaluated your query thoroughly you...
256914,Is Minoxin 5% lotion advisable instead of Foli...,"Hi, i am 25 year old girl, i am having massive...",Hello and Welcome to ‘Ask A Doctor’ service.I ...


In [6]:
!pip install -q auto-gptq
!pip install -q optimum
!pip install -q bitsandbytes

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [9]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [10]:
# Set up BitsAndBytesConfig for 4-bit quantization
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
from huggingface_hub import login
from transformers import AutoModelForCausalLM

# Log in to your Hugging Face account
login()

In [12]:
# Model name
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# Load the model with the 4-bit quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=False,
    revision="main",
    quantization_config=nf4_config  # Use the NF4 quantization configuration
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [14]:
# craft prompt
patient = '''
Hello doctor,I am having an uncomfortable feeling in between the middle of my spine and left shoulder blade. It seems to get stiff, and my bones pop a lot around there, and it is 
very uncomfortable to sit in specific ways feels like my bones shift, well the other night it seemed as if my shoulder would pop out of place causing more stiffness after a while of
dealing with it. I ended up feeling a popping sensation, where the stiffness was an instant relief. It was so shocking it put my anxiety through the roof. I thought I was going to 
die because my whole body was shaking and tingling. What am I dealing with? Right now I have no pain or tender near that area still relief but curious on what happened and why I got 
this shock and that popping feeling so loud it scared me half to death.
'''
prompt=f'''[INST] {patient} [/INST]'''

In [15]:
instruction_string = f"""MediGPT is designed to provide accurate and helpful responses based on the medical information or queries that you provide. \
MediGPT aims to offer clear, concise, and reliable explanations about medical conditions, treatments, and health-related topics.

Please ask any medical or health-related question, and MediGPT will generate a response based on available knowledge."""


prompt_template = lambda patients: f'''<s>[INST] {instruction_string} \n{patients} \n[/INST]</s>''' # Changed intstruction_string to instruction_string

# Pass 'patient' to prompt_template instead of 'patients'
prompt = prompt_template(patient)
print(prompt)

<s>[INST] MediGPT is designed to provide accurate and helpful responses based on the medical information or queries that you provide. MediGPT aims to offer clear, concise, and reliable explanations about medical conditions, treatments, and health-related topics.

Please ask any medical or health-related question, and MediGPT will generate a response based on available knowledge. 

Hello doctor,I am having an uncomfortable feeling in between the middle of my spine and left shoulder blade. It seems to get stiff, and my bones pop a lot around there, and it is 
very uncomfortable to sit in specific ways feels like my bones shift, well the other night it seemed as if my shoulder would pop out of place causing more stiffness after a while of
dealing with it. I ended up feeling a popping sensation, where the stiffness was an instant relief. It was so shocking it put my anxiety through the roof. I thought I was going to 
die because my whole body was shaking and tingling. What am I dealing wit

In [17]:
def _create_data(Patient, Doctor):
    return f'''<s>[INST] {instruction_string} \n{Patient} \n[/INST]"{Doctor}"</s>'''

In [18]:
data_for_llm_training = [_create_data(patient, doctor) for patient, doctor in zip(df['Patient'], df['Doctor'])] # Use patient and doctor as loop variables instead of global variables Patient and Doctor

In [19]:
data_for_llm_training[0]

'<s>[INST] MediGPT is designed to provide accurate and helpful responses based on the medical information or queries that you provide. MediGPT aims to offer clear, concise, and reliable explanations about medical conditions, treatments, and health-related topics.\n\nPlease ask any medical or health-related question, and MediGPT will generate a response based on available knowledge. \nHi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear? \n[/INST]"Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->"</s>'

In [20]:
trainset = df[:60]
testset = df[60:80]

In [21]:
from datasets import DatasetDict, Dataset

In [23]:
data_dict_train = {'examples':trainset}
data_dict_test = {'examples':testset}

In [24]:
data1 = DatasetDict()

In [25]:
data1['train'] = Dataset.from_dict(data_dict_train)
data1['test'] = Dataset.from_dict(data_dict_test)

In [26]:
data1

DatasetDict({
    train: Dataset({
        features: ['examples'],
        num_rows: 3
    })
    test: Dataset({
        features: ['examples'],
        num_rows: 3
    })
})

In [27]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["examples"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data1.map(tokenize_function, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [28]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [31]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 3

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "medigpt-ft-mistral",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [33]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import Trainer, TrainingArguments

# Define LoRA configuration
lora_config = LoraConfig(
    r=16,                        # Rank of the decomposition matrix
    lora_alpha=32,                # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Layers to which LoRA applies
    lora_dropout=0.05,            # Dropout probability
    task_type=TaskType.CAUSAL_LM  # Task type (Causal Language Modeling)
)

# Attach LoRA adapters to the quantized model
model = get_peft_model(model, lora_config)

# Disable caching to avoid warnings
model.config.use_cache = False

# Set up the Trainer with the modified model
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Re-enable cache (for inference)
model.config.use_cache = True


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.047900,8.191407
2,2.047900,8.191407
3,2.047900,8.191407


In [35]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
patients = '''
    Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for annular bulging and tear?
'''
patients1 = '''
    Hi doctor, I am a 22-year-old female who was diagnosed with hypothyroidism (genetic) when I was 12. Over the past five years, I have become around 50
    pounds overweight and all of my attempts to lose have seemed to fail so I have given up, but my weight has stayed the same. There is so much information
    put there about losing weight with hypothyroidism but it all seems to conflict. I am so unsure as to what type of exercise and diet I should follow as a 
    result but I still would like to lose weight, but most importantly have my body feel better. What can I do? I am currently on Levothyroxine, Buspar, 
    and Benedryl.
'''

prompt_template = lambda patients: f'''<s>[INST] {instruction_string} \n{patients} \n[/INST]</s>'''

prompt = prompt_template(patients1)

# prompt=f'''<s>[INST] {intstructions_string} \n{news1} \n[/INST]</s>'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

In [36]:
# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                         attention_mask = inputs['attention_mask'].to('cuda'),
                        #  pad_token_id=tokenizer.eos_token_id,
                         max_new_tokens=100)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [37]:
print(tokenizer.batch_decode(outputs)[0])

<s><s>[INST] MediGPT is designed to provide accurate and helpful responses based on the medical information or queries that you provide. MediGPT aims to offer clear, concise, and reliable explanations about medical conditions, treatments, and health-related topics.

Please ask any medical or health-related question, and MediGPT will generate a response based on available knowledge. 

    Hi doctor, I am a 22-year-old female who was diagnosed with hypothyroidism (genetic) when I was 12. Over the past five years, I have become around 50
    pounds overweight and all of my attempts to lose have seemed to fail so I have given up, but my weight has stayed the same. There is so much information
    put there about losing weight with hypothyroidism but it all seems to conflict. I am so unsure as to what type of exercise and diet I should follow as a 
    result but I still would like to lose weight, but most importantly have my body feel better. What can I do? I am currently on Levothyroxine,

In [38]:
prompt

'<s>[INST] MediGPT is designed to provide accurate and helpful responses based on the medical information or queries that you provide. MediGPT aims to offer clear, concise, and reliable explanations about medical conditions, treatments, and health-related topics.\n\nPlease ask any medical or health-related question, and MediGPT will generate a response based on available knowledge. \n\n    Hi doctor, I am a 22-year-old female who was diagnosed with hypothyroidism (genetic) when I was 12. Over the past five years, I have become around 50\n    pounds overweight and all of my attempts to lose have seemed to fail so I have given up, but my weight has stayed the same. There is so much information\n    put there about losing weight with hypothyroidism but it all seems to conflict. I am so unsure as to what type of exercise and diet I should follow as a \n    result but I still would like to lose weight, but most importantly have my body feel better. What can I do? I am currently on Levothyro

In [39]:
model.config.forced_eos_token_id

In [40]:
model.save_pretrained("medigpt-ft-mistral")